In [2]:
import pickle
import numpy as np
import torch


dataset = 'cifar10'
cpt_num = 1000

with open(f'concept_bank/concept_bank_num_4167.pkl', 'rb') as file:
    Cpt_base = pickle.load(file)
with open(f'concept_bank/{dataset}/{dataset}_image_representation_train.pkl', 'rb') as file:
    M_img = pickle.load(file)

concept_name = list(Cpt_base.keys())
M_cpt = torch.stack(list(Cpt_base.values())).cuda()
M_img = torch.from_numpy(M_img).cuda()
Rough_Selection = {}

while len(Rough_Selection) < cpt_num:
    Sim = torch.matmul(M_img, M_cpt.T)
    Var = torch.var(Sim, dim=0)
    slct_cpt_idx = torch.argmax(Var).item()
    slct_cpt_name = concept_name[slct_cpt_idx]
    slct_cpt_embed = M_cpt[slct_cpt_idx]
    Rough_Selection[slct_cpt_name] = slct_cpt_embed.cpu()

    M_cpt = torch.cat((M_cpt[:slct_cpt_idx], M_cpt[slct_cpt_idx+1:]))
    concept_name.remove(slct_cpt_name)
    M_slct = torch.matmul(torch.matmul(M_img, slct_cpt_embed).unsqueeze(1), slct_cpt_embed.unsqueeze(0))
    M_img = M_img - M_slct

with open(f'concept_bank/{dataset}/{dataset}_rough_selection_bar_3.pkl', 'wb') as file:
    pickle.dump(Rough_Selection, file)